In [10]:
#packages import
import requests as rq
import numpy as np
import pandas as pd
import datetime as dt
import os
import re
import time

In [11]:
#requests url and content
req = rq.get('https://www.118712.fr/annuaire/ville/roubaix-59/assurance-professionnelle')
page_content = req.content

In [12]:
#BeautifulSoup
from bs4 import BeautifulSoup
bs_content = BeautifulSoup(page_content, 'html.parser')
bs_content

<!DOCTYPE html>
<html lang="en-US"><head><title>Just a moment...</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="noindex,nofollow" name="robots"/><meta content="width=device-width,initial-scale=1" name="viewport"/><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}@media (prefers-color-scheme:dark){body{background-color:#222;color:#d9d9d9}body a{color:#fff}body a:hover{color:#ee730a;text-decoration:underline}body .lds-ring div{border-color:#999 transparent transparent}body .font-red{color:#b20f03}body .big-button,body .pow-button{background-color:#4693ff;color:#1d1d1d}body #challenge-success-text{background-image:url()}body #challenge-error

In [13]:
#dynamic content -> Selenium
import selenium
from selenium import webdriver
#submodules and classes within selenium.webdriver.common
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
#manage exceptions tracking with common classes
from selenium.common.exceptions import NoSuchWindowException , TimeoutException, WebDriverException, NoSuchElementException

#set up chrome webdriver classes
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

#set up Webdriverwait and support classes
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [74]:
#request
try:
    #Initialize Webdriver
    service = Service()
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    driver.get('https://www.118712.fr/recherche/auto/roubaix/assurance-professionnelle')

     

    #click PrivacyCompliance
    # button_Ok = driver.find_element(By.CSS_SELECTOR, '#privacy-cookie-banner__privacy-accept')
    # button_Ok.click()

    #find and count 'next page buttons' to define how many pages are to scrap
    buttons = driver.find_elements(By.CSS_SELECTOR, 'button.len1.adpJam')
    nbPage = 0

    #loop in buttons list to check if onclick attrbutes match with 'next buttons'
    for button in buttons:
        onclick_attribute = button.get_attribute('onclick')
        if onclick_attribute:
            pattern=r'changePageUseCurrentBounds\(\d+\)'
            match = re.search(pattern, onclick_attribute)
            # sum nb of pages
            if match:
                nbPage +=1
            else:
                continue
        else:
            continue
        
     #create empty lists in wich append informations as names(titles) and street address
    titles_all=[]
    addresses_all = []
    cities_all = []

    #loop through page results 

    for page in range(1,(nbPage+2)*2):

        # Wait for the elements to be visible
        elements_loaded = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2.h4')))
       # Fetch and print the text of the elements
        titles = [h2Elt.text   for h2Elt in elements_loaded]
        #print(titles)
        titles_all.append(titles)

        # Fetch and print the text of the address
        elements_addresses = driver.find_elements(By.CSS_SELECTOR, 'article > address.bi_adress.txt_md > p:first-child' )

        addresses = [address.text for address in elements_addresses]
        addresses_all.append(addresses)

        elements_cities = driver.find_elements(By.CSS_SELECTOR, 'article > address.bi_adress.txt_md > p:nth-child(2)' )
        cities = [citie.text for citie in elements_cities]
        cities_all.append(cities)
        
    #find phone numbers, displayed after clicking a first button
        #locate buttons 'fantomas' to click
        buttons_tobeclicked_located = driver.find_elements(By.CSS_SELECTOR, 'section.bi_list > article.bi.adp-map-item  > div.bi_cta > a.button.primary.bi_fantomas')
        
    #create an ampty list to add list numbers

        tel_nb_list = []
        for button in buttons_tobeclicked_located:
        
        # Move to the button before clicking with 'ActionChains' webdriver submodule ans classes
            actions = ActionChains(driver)
            actions.move_to_element(button).perform()
            #click button to display phone numbe
            button.click()
            # driver.implicitly_wait(2)
    
            #wait display and locate buttons mentionin tel number, dispayed after clicking initial button
            button_tel_located = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section.bi_list > article.bi.adp-map-item > div.bi_cta > a.button.primary.ab_test')))

            #collect list of "a" html elt      
            links = driver.find_elements(By.CSS_SELECTOR, 'a.button.primary.ab_test')

            #loop in buttons list to extract href attribute containing tel number
            for link in links: 
                tel_href = link.get_attribute('href')
                tel_nb_list.append(tel_href)

# click 'next button' by HTML id attribute ()
        next_button = driver.find_element(By.CSS_SELECTOR, 'button.len1.adpJam')
        next_button.click()

        # Optional: Add a delay to ensure the page fully loads before extracting results
        # time.sleep(2)

#manage alle possible exceptiosn
except NoSuchWindowException as NsWe:
    print(f"exception handled: {NsWe}")
except TimeoutException as te:
    print(f"TimeoutException: {te}")
    # Handle timeout exception
except WebDriverException as wde:
    print(f"WebDriverException: {wde}")
    # Handle WebDriver related exception
except NoSuchElementException as nse:
    print(f"NoSuchElementException: {nse}")
    # Handle NoSuchElementException
except Exception as ex:
    print(f"An unexpected exception occurred: {ex}")
    # Handle other exceptions

#close driver
finally:
    # Close the browser window in the finally block to ensure it's closed even if an exception occurs
    if 'driver' in locals() and driver is not None:
        driver.quit()


In [75]:
# If you want a flat list containing titles from all pages, use extend
flat_titles_list = [title for titles_page in titles_all for title in titles_page]
print(flat_titles_list) 
flat_addresses_list = [address for addresses_page in addresses_all for address in addresses_page]
print(flat_addresses_list)  
flat_cities_list = [citie for cities_page in cities_all for citie in cities_page]
print(flat_cities_list)  
print(tel_nb_list)

['Matmut', 'Macif assurances', 'Cap catoire assurances prevoyance', 'Swiss life', 'Allianz delesalle christian agent général', 'Gesco assurances santé', 'Ox6gen assurances', 'Gmf assurances', 'Cabinet toupet', 'Ag sante', 'Axa chapat olivier agent général', 'Swisslife cabinet kleas', 'Trinel antoine', 'Mutuelle smh', 'Assu 2000', 'Monceau assurances', 'Bwa - gaétan waterlot assurances', 'Akor2prêt', 'Mma efficiences assurances agents généraux', 'Grebert stephanie eirl', 'Mercer france', 'Hue bertrand', 'Agence camille fontaine axa', 'Gan assurances roger joël agent général', 'Axa bruno charlet', 'Allianz breant francois', 'Agence axa pannecoucke', 'Swisslife laurence dubois agent général', 'Allianz breant françois agent général', 'Maif assurances lens', 'Bwa - gaétan waterlot assurances', 'Akor2prêt', 'Monceau assurances', 'Grebert stephanie eirl', 'Sébastien vandewalle assurances', 'Mma efficience assurances agents généraux', 'Agence camille fontaine axa', 'Maif assurances lambersart'

In [110]:
#define the number of rows and only use the half because iteratio on the double of page number + flip rows because seems to be better fitted on the second half
max_rows = (len(flat_titles_list)) 
flat_titles_list = np.array(flat_titles_list)[::-1]
flat_titles_list = flat_titles_list[:max_rows]
flat_addresses_list = np.array(flat_addresses_list)[::-1]
flat_addresses_list = flat_addresses_list[:max_rows]
flat_cities_list = np.array(flat_cities_list)[::-1]
flat_cities_list = flat_cities_list[:max_rows]
tel_nb_list = np.array(tel_nb_list)[::-1]
tel_nb_list = tel_nb_list[:max_rows]

In [149]:
insurers_list2 = {'insurers': flat_titles_list,
                 'address' : flat_addresses_list,
                 'city': flat_cities_list,
                 'tel': tel_nb_list}
    
insurers_list2 = pd.DataFrame.from_dict(insurers_list2)
insurers_list2

,insurers,address,city,tel
0,La france mutualiste,41 Bd Carnot,59800 Lille,tel:+333 62 72 59 00
1,Agence camille fontaine axa,19 Chem Dais,59112 Carnin,tel:+336 77 84 22 37
2,Maif assurances arras,30 Av Paul Michonneau,62000 Arras,tel:+333 21 60 66 66
3,Swisslife laurence dubois agent général,2 R Jean Jaurès,59950 Auby,tel:+333 27 92 15 15
4,Allianz breant françois agent général,44 R Jules Guesde,62410 Wingles,tel:+333 21 37 13 63
...,...,...,...,...
190,Allianz delesalle christian agent général,43 R Mar Foch,59100 Roubaix,tel:+333 20 22 33 90
191,Swiss life,1 R Mar De Lattre De Tassigny,59100 Roubaix,tel:+333 21 40 78 30
192,Cap catoire assurances prevoyance,24 Bd Belfort,59100 Roubaix,tel:+333 21 57 36 76
193,Macif assurances,50 Grande Rue,59100 Roubaix,tel:+333 21 24 70 15


In [246]:
# insurers_list2[['zip_code', 'city']] = insurers_list2['city'].str.split(' ', n=1, expand=True)
# insurers_list2
# Split the 'city' column into 'zip code' and 'city' columns
insurers_list2[['zip code', 'city']] = insurers_list2['city'].str.split(' ', n=1, expand=True)

# Display the updated DataFrame
print(insurers_list2)


ValueError: Columns must be same length as key

In [235]:
insurers_list2_cities_Spl = insurers_list2_cities[['zip code', 'city']] = insurers_list2['city'].str.split(' ', n=1, expand=True)
col_mask = ['zip code', 'city']
insurers_list2_cities = insurers_list2_cities.loc[:, col_mask]
insurers_list2_cities = insurers_list2_cities.drop_duplicates(subset='zip code')


ValueError: Columns must be same length as key

In [220]:
insurers_list2_cities_mask = ["Lille", "Lambersart", "Villeneuve D'Ascq", "Wasquehal", "Roubaix"]
mask = insurers_list2['city'].isin(insurers_list2_cities_mask )

In [213]:
insurers_list2.loc[mask]

,insurers,address,city,tel


In [106]:
import pandas as pd

# Assuming insurers_list2 is a dictionary-like object
# Convert it to a DataFrame
insurers_list2_df = pd.DataFrame.from_dict(insurers_list2)

# Group the DataFrame by the 'city' column and apply the all() function
insurers_list2_grouped = insurers_list2_df.groupby('city').tail()

# Print or use the resulting DataFrame
print(insurers_list2_grouped)


                                      insurers                        address  \
6            Maif assurances villeneuve d'ascq                    200 Bd Mons   
19   Mma efficience assurances agents généraux                       R Boilly   
20                     Allianz breant francois               40 R Sadi Carnot   
25     Gan assurances roger joël agent général          9 R Florimond Lemaire   
26   Mma efficience assurances agents généraux                  2 Pl Mar Foch   
..                                         ...                            ...   
190  Allianz delesalle christian agent général                  43 R Mar Foch   
191                                 Swiss life  1 R Mar De Lattre De Tassigny   
192          Cap catoire assurances prevoyance                  24 Bd Belfort   
193                           Macif assurances                  50 Grande Rue   
194                                     Matmut                  19 Grande Rue   

                        cit

In [79]:
import pandas as pd

# Assuming insurers_list2 is a DataFrame

# Create a boolean mask based on whether the 'city' column ends with 'Roubaix'
mask = insurers_list2['city'].str.endswith('baix')

# Filter the DataFrame using boolean indexing
filtered_insurers_list2 = insurers_list2[mask]

# Print or use the filtered DataFrame
print(filtered_insurers_list2)


                                      insurers                        address  \
189                     Gesco assurances santé      57 Av Jean Baptiste Lebas   
190  Allianz delesalle christian agent général                  43 R Mar Foch   
191                                 Swiss life  1 R Mar De Lattre De Tassigny   
192          Cap catoire assurances prevoyance                  24 Bd Belfort   
193                           Macif assurances                  50 Grande Rue   
194                                     Matmut                  19 Grande Rue   

              city                   tel  
189  59100 Roubaix  tel:+333 27 81 22 23  
190  59100 Roubaix  tel:+333 20 22 33 90  
191  59100 Roubaix  tel:+333 21 40 78 30  
192  59100 Roubaix  tel:+333 21 57 36 76  
193  59100 Roubaix  tel:+333 21 24 70 15  
194  59100 Roubaix  tel:+333 21 22 38 38  


In [38]:
insurers_list2_noDup_f = insurers_list2.drop_duplicates(keep='last')
insurers_list2_noDup_f

,insurers,address,city,tel
67,Sébastien vandewalle assurances,37 Av Fernand Lobbedez,62000 Arras,tel:+333 20 92 98 02
68,Daniel gallez agent general areas,7 R Paul Doumer,62000 Arras,tel:+333 21 68 68 00
69,Maif assurances arras,30 Av Paul Michonneau,62000 Arras,tel:+333 20 97 68 54
70,Allianz breant françois agent général,44 R Jules Guesde,62410 Wingles,tel:+333 91 20 44 71
71,Maif assurances lambersart,410 Av Dunkerque,59130 Lambersart,tel:+333 28 48 50 00
72,Akor2prêt,66 Rte Lens,62223 Sainte Catherine D'Arras,tel:+333 21 22 38 38
74,Bwa - gaétan waterlot assurances,12 Bd Liberté,62000 Arras,tel:+333 21 24 70 15
75,Swisslife laurence dubois agent général,2 R Jean Jaurès,59950 Auby,tel:+333 27 92 15 15
76,Grebert stephanie eirl,217 R Villars,59220 Denain,tel:+333 27 44 03 22
77,Mercer france,20 All Château Blanc,59290 Wasquehal,tel:+339 69 32 31 05
